# Load Dataset

In [182]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

dataset=pd.read_excel("C:/Users/10253/Desktop/ML-CW-2/Data/trainDataset.xls")
dataset.head(len(dataset)-1)

,ID,pCR (outcome),RelapseFreeSurvival (outcome),Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,TRG002174,1,144.000000,41.0,0,0.0,0.0,1.0,3,3,...,0.517172,0.375126,3.325332,0.002314,3.880772e+06,473.464852,0.000768,0.182615,0.030508,0.000758
1,TRG002178,0,142.000000,39.0,1,1.0,0.0,0.0,3,3,...,0.444391,0.444391,3.032144,0.005612,2.372010e+06,59.459710,0.004383,0.032012,0.001006,0.003685
2,TRG002204,1,135.000000,31.0,0,0.0,0.0,1.0,2,1,...,0.534549,0.534549,2.485848,0.006752,1.540027e+06,33.935384,0.007584,0.024062,0.000529,0.006447
3,TRG002206,0,12.000000,35.0,0,0.0,0.0,1.0,3,3,...,0.506185,0.506185,2.606255,0.003755,6.936741e+06,46.859265,0.005424,0.013707,0.000178,0.004543
4,TRG002210,0,109.000000,61.0,1,0.0,0.0,0.0,2,1,...,0.462282,0.462282,2.809279,0.006521,1.265399e+06,39.621023,0.006585,0.034148,0.001083,0.005626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,TRG002954,0,49.250000,34.3,0,0.0,0.0,1.0,3,3,...,0.418382,0.418382,2.995603,0.004243,1.005061e+06,156.627179,0.002228,0.136015,0.022148,0.002098
395,TRG002955,1,49.250000,46.1,0,0.0,0.0,1.0,3,3,...,0.439568,0.439568,3.056046,0.001339,1.671271e+07,79.989003,0.003282,0.024716,0.000812,0.003078
396,TRG002958,0,48.500000,53.3,0,0.0,0.0,1.0,2,1,...,0.527779,0.527778,1.500000,0.003728,2.132007e+05,0.996746,0.252582,0.007380,0.000037,0.231059
397,TRG002961,0,47.500000,68.8,1,0.0,0.0,0.0,3,3,...,0.313693,0.313693,3.573557,0.001112,2.008034e+07,204.864200,0.001372,0.054063,0.003697,0.001368


In [183]:
# from sklearn.impute import  SimpleImputer
# imp=SimpleImputer(missing_values=np.nan,strategy='mean')
# names=dataset.columns.drop('ID')
# a=imp.fit_transform(dataset.drop('ID',axis=1))
# imp=SimpleImputer(missing_values=999,strategy='mean')
# a=imp.fit_transform(a)
# df=pd.DataFrame(a,columns=
#                 names)
# df

# Data Preprocessing

### Removing duplicate

In [184]:
dataset.drop_duplicates(inplace=True) #because with 119 features, it is unlikely that exact same data happens in real world, it is useless for training and testing; hence remove it before dataset split

### Split Dataset

In [185]:
dataset['pCR (outcome)'].value_counts()

0      299
1       96
999      5
Name: pCR (outcome), dtype: int64

In [186]:
y=dataset['pCR (outcome)']
X=dataset.drop(['pCR (outcome)'],axis=1)


X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

### drop columns that are not applicable

In [187]:
class columnDropperTransformer(): #https://stackoverflow.com/questions/68402691/adding-dropping-column-instance-into-a-pipeline
    def __init__(self,columns):
        self.columns=columns

    def fit(self,X,y=None):
        return self

    def transform(self,X,y=None):
        return X.drop(self.columns,axis=1)

    def fit_transform(self,X,y=None):
        return X.drop(self.columns,axis=1)

dropper = columnDropperTransformer(['ID'])
X_train=dropper.fit_transform(X_train)

X_test=dropper.transform(X_test)

### missing data

In [188]:
dataset.isnull().sum()

ID                               0
pCR (outcome)                    0
RelapseFreeSurvival (outcome)    0
Age                              0
ER                               0
                                ..
original_ngtdm_Busyness          0
original_ngtdm_Coarseness        0
original_ngtdm_Complexity        0
original_ngtdm_Contrast          0
original_ngtdm_Strength          0
Length: 120, dtype: int64

In [189]:
from sklearn.impute import SimpleImputer

names=X_train.columns

imp=SimpleImputer(missing_values=np.nan,strategy='mean')
# X_train[:]=imp.fit_transform(X_train)
# X_test[:]=imp.transform(X_test)
X_train=imp.fit_transform(X_train)
X_test=imp.transform(X_test)

imp.missing_values=np.float64(999)
# X_train[:]=imp.fit_transform(X_train)
# X_test[:]=imp.transform(X_test)
X_train=imp.fit_transform(X_train)
X_test=imp.transform(X_test)

### Removing outliers

In [190]:
from sklearn.neighbors import LocalOutlierFactor

# because use LocalOutlierFactor may lose useful information becuase whenever there is an outlier in any features of the sample, the sample will be detected as outlier sample and all its information will be set to 999
# therefore, use other method

class outlierHandlingWithDev():
    def fit(self,X,y=None):
        self.mean=np.zeros(len(X[0]))
        self.std=np.zeros(len(X[0]))
        for i in range(len(X[0])):
            self.mean[i]=X[:,i].mean()
            self.std[i]=X[:,i].std()

        return self

    def transform(self,X,y=None):
        for i in range(len(X)):
            for j in range(len(X[i])):
                if np.abs(X[i][j]-self.mean[j])>3*self.std[j]:
                    X[i][j]=999
        return X

    def fit_transform(self,X,y=None):
        self.fit(X,y)
        return self.transform(X,y)
    # def fit(self,X,y=None):
    #     self.mean=X.mean(axis=0)
    #     self.std=X.std(axis=0)
    #
    #     return self
    #
    # def transform(self,X,y=None):
    #     for i in X.columns:
    #         for j in range(len(X)):
    #             if abs(X[i][j]-self.mean[i])>(3*self.std[i]):
    #                 X[i][j]=999
    #
    #     return X
    #
    # def fit_transform(self,X,y=None):
    #     self.fit(X,y)
    #     return self.transform(X,y)

detector=outlierHandlingWithDev()
detector.fit(X_train)
detector.transform(X_train)
detector.transform(X_test)

X_train=imp.fit_transform(X_train)
X_test=imp.transform(X_test)

X_train=pd.DataFrame(X_train,columns=names)
X_test=pd.DataFrame(X_test,columns=names)

### Binning

### resample

### MinMaxScaler

In [191]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X_train[:]=scaler.fit_transform(X_train)
X_test[:]=scaler.transform(X_test)
X_train

,RelapseFreeSurvival (outcome),Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,LNStatus,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,0.432836,0.141335,0.0,0.0,0.0,1.0,1.0,0.5,0.0,0.0,...,1.000000,0.481086,0.621166,0.141300,0.037519,0.099893,0.001924,0.335313,0.217233,0.006973
1,0.440299,0.424006,0.0,0.0,1.0,0.0,0.5,0.0,0.0,1.0,...,0.483330,0.507451,0.797838,0.198167,0.215474,0.209591,0.000661,0.130973,0.019532,0.002527
2,0.544776,0.357800,0.0,0.0,1.0,0.0,0.5,0.0,0.0,0.0,...,0.624275,0.655431,0.268012,0.087400,0.067762,0.002181,0.078413,0.009931,0.000081,0.239658
3,0.208955,0.284437,0.0,0.0,1.0,0.0,1.0,0.5,0.0,0.0,...,0.473587,0.497222,0.743269,0.387069,0.018754,0.191705,0.000886,0.524509,0.325063,0.003327
4,0.223881,0.729643,0.0,0.0,0.0,1.0,0.5,0.5,0.0,1.0,...,0.507146,0.532455,0.598848,0.260036,0.020785,0.016987,0.010094,0.087847,0.007427,0.031514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,0.044776,0.247337,1.0,0.0,0.0,0.0,1.0,0.5,0.0,1.0,...,0.467486,0.490817,0.622367,0.035246,0.855396,0.024214,0.006928,0.022442,0.000572,0.023740
276,0.067164,0.175544,0.0,1.0,0.0,0.0,0.5,0.0,0.0,0.0,...,0.648996,0.681385,0.368009,0.098513,0.070424,0.097305,0.001874,0.274368,0.136347,0.007120
277,0.597015,0.730196,1.0,0.0,0.0,0.0,0.5,0.0,0.0,1.0,...,0.000014,0.000015,0.214130,0.071263,0.013972,0.000000,0.025685,0.000000,0.000000,0.000000
278,0.029851,0.514420,1.0,1.0,0.0,0.0,0.5,0.0,0.0,1.0,...,0.363633,0.381781,1.000000,0.182714,0.120466,0.364836,0.000339,0.353319,0.143804,0.001626


### Feature Selection

In [193]:
from sklearn.feature_selection import SelectKBest
selector=SelectKBest(k=50)
X_train=selector.fit_transform(X_train,y_train)
X_test=selector.transform(X_test)

### PCA

In [11]:

# c=dataset.corr().abs()
# s=c.unstack()
# so=s.sort_values(kind="quicksort")





# delcol=[]
# c=dataset.corr().shape[0]

# names=dataset.corr().columns
#
# for i in range(c):
#     for j in range(i+1,c):
#         if dataset.corr().iloc[i,j]>0.8 :
#             delcol.append((names[i],names[j]))
# print(len(delcol))
#
# import numpy as np
# np.a
#
# for i in range(c):
#     count = 0
#     for j in range(c):
#         if dataset.corr().iloc[i,j]>0.8:
#             count += 1





,0,1,2,3,4,5,6,7,8,9,...,108,109,110,111,112,113,114,115,116,117
0,50.000000,55.956194,0.0,0.0,0.0,1.0,3.0,2.0,1.0,1.0,...,0.212197,0.212197,3.277613,0.000692,1.450165e+07,342.470452,0.000971,0.129494,0.019639,0.000884
1,86.000000,69.000000,0.0,0.0,1.0,0.0,2.0,1.0,1.0,1.0,...,0.550507,0.550507,2.573220,0.005959,1.052641e+06,21.346044,0.012082,0.024663,0.000567,0.010516
2,31.000000,57.400000,0.0,0.0,0.0,1.0,3.0,3.0,1.0,0.0,...,0.122225,0.122225,1.921928,0.003720,1.665694e+05,36.235243,0.010167,0.146358,0.026189,0.009567
3,68.000000,62.000000,0.0,0.0,0.0,1.0,2.0,1.0,1.0,1.0,...,0.661168,0.661168,1.798795,0.001021,1.426398e+07,11.358284,0.022155,0.005759,0.000026,0.021147
4,75.583333,42.700000,0.0,0.0,0.0,1.0,3.0,3.0,1.0,0.0,...,0.374829,0.374829,3.201460,0.004586,1.180094e+06,53.909560,0.005069,0.062390,0.003892,0.004351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,43.000000,55.000000,1.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,...,0.661700,0.661700,1.672933,0.001252,6.303287e+06,7.163648,0.035187,0.006469,0.000038,0.032914
276,61.000000,48.000000,0.0,0.0,0.0,1.0,2.0,1.0,1.0,1.0,...,0.151443,0.151443,3.175123,0.002063,2.963878e+06,123.448706,0.002424,0.100139,0.011008,0.002154
277,56.083333,46.300000,0.0,0.0,0.0,1.0,2.0,2.0,1.0,0.0,...,0.550730,0.550730,2.176353,0.004691,1.335551e+06,11.074256,0.022918,0.013204,0.000154,0.019456
278,55.083333,58.900000,0.0,0.0,0.0,1.0,3.0,3.0,1.0,0.0,...,0.235291,0.233849,2.947703,0.000975,5.464581e+06,205.821528,0.001657,0.130725,0.020401,0.001553


# Feature Preprocession

# Methods